# PCA + Neural Network Classifier with MNIST


## Libraries

In [ ]:
# libraries
#using Flux              # the julia ml library
using Images            # image processing and machine vision for julia
using MLJ               # make_blobs, rmse, confmat, f1score, coerce
using MLJFlux           # NeuralNetworkClassifier, CUDALibs
using MLDataUtils       # label, nlabel, labelfreq
using MLDatasets        # mnist

#using LinearAlgebra     # pinv pseudo-inverse matrix
#using Metrics           # r2-score
using Random
using StatsBase         # standardize (normalization)
#using Distributions

using Plots; gr()
#using StatsPlots
using Printf

#using CSV
using DataFrames


## Functions

In [ ]:
# metrics
function printMetrics(ŷ, y)
    display(confmat(ŷ, y))
    println("accuracy: ", round(accuracy(ŷ, y); digits=3))
    println("f1-score: ", round(multiclass_f1score(ŷ, y); digits=3))
end


In [ ]:
# lib functions
image2Vector(M) = vec(Float64.(M))

function batchImage2Vector(imagesArray3D)
    h, v, N = size(imagesArray3D)
    vectorOfImageVectors = [ image2Vector( imagesArray3D[:, :, i] ) for i in 1:N]
end

function batchImage2DF(imagesArray3D)
    vectorOfImageVectors = batchImage2Vector(imagesArray3D)
    M = reduce(hcat, vectorOfImageVectors)
    DataFrame(M', :auto)
end


## Loading the data

In [ ]:
# load mnist from MLDatasets
trainX_original,      trainY_original      = MNIST.traindata()
validationX_original, validationY_original = MNIST.testdata();

display([MNIST.convert2image(MNIST.traintensor(i)) for i in 1:5])
trainY_original[1:5]'

In [ ]:
# split trainset, testset, validation set
Random.seed!(1)
(trainX, trainY), (testX, testY) = stratifiedobs((trainX_original, trainY_original), p = 0.7)
validationX = copy(validationX_original); validationY = copy(validationY_original)

size(trainX), size(testX), size(validationX)

## Data preprocessing

Data preprocessing depends on the data source, thus can widely vary from what is shown here.

In [ ]:
function preprocessing(X, y)
    newX = batchImage2DF(X)
    #coerce!(newX)   # no need, all scitypes are Continuous in this example
    new_y = coerce(y, OrderedFactor)
    
    return (newX, new_y)
end

X, y = preprocessing(trainX, trainY);

In [ ]:
scitype(X)

In [ ]:
scitype(y)

## Training, Testing, Validation

### Load and pipe algorithms

In [ ]:
# reduce predictors
PCA = @load PCA pkg=MultivariateStats verbosity=0
reducer = PCA(pratio = 0.95)

In [ ]:
# standardize predictors
std = Standardizer();

In [ ]:
# nnet
NeuralNetworkClassifier = @load NeuralNetworkClassifier pkg=MLJFlux verbosity=0
nnet = NeuralNetworkClassifier(acceleration=CUDALibs())

In [ ]:
pipe = @pipeline reducer std nnet

### Create and train the machine


In [ ]:
pipe.neural_network_classifier.epochs=1
mach = MLJ.machine(pipe, X, y) |> fit!;

In [ ]:
MLJ.save("mnist_pca_nn_machine_1.jlso", mach)


In [ ]:
fitted_params(mach).neural_network_classifier.chain

In [ ]:
losses = report(mach).neural_network_classifier.training_losses
epochs = pipe.neural_network_classifier.epochs
plot(0:epochs, losses, title="Error function", size=(500,300), linewidth=2, legend=false)
xlabel!("Epochs")
ylabel!("Cross-entropy loss")

In [ ]:
display(minimum(losses))
best_epoch = argmin(losses) - 1

In [ ]:
# tuning single hyper-parameter
r = range(pipe, :(neural_network_classifier.lambda), lower=0.01, upper=1.0, scale=:log)

self_tuning_pipe = TunedModel(model=pipe,
                              resampling=CV(nfolds=2),
                              tuning=Grid(resolution=10),
                              range=r,
                              measure=cross_entropy,
                              acceleration=CPUProcesses(),
                              acceleration_resampling=CPUThreads())

In [ ]:
mach = machine(self_tuning_pipe, X, y) |> fit!


In [ ]:
ŷ = predict_mode(mach, X)
ŷ[1:5]

In [ ]:
printMetrics(ŷ, y)